In [34]:
import pandas as pd
import numpy as np
from analysis.analysis import create_analysis
from analysis.pipeline import create_pipeline
from analysis.stages import FormatData, NumericalSummary, CategoricalSummary, ClassifyColumns



In [36]:
test_data = {
    "scores": [2, 3, 0, 2, 2, 2, 1, 2],
    "reactionTimes": [
        {"clickTimes": [1008, 381, 551, 266]},
        {"clickTimes": [1007, 999, 499, 245]},
        {"clickTimes": [897, 787, 300, 456]},
        {"clickTimes": [978, 654, 321, 111]},
        {"clickTimes": [700, 800, 600, 200]},
        {"clickTimes": [1500, 500, 700, 100]},
        {"clickTimes": [1400, 1300, 800, 900]},
        {"clickTimes": [1234, 5678, 4321, 8765]},
    ],
    "reactionTimesGlow": [
        [1348, 1961],
        [1546, 1460],
        [1423, 887, 1186],
        [1423, 111],
        [700],
        [3702],
        [1500, 1200, 1300, 1400],
        [900],
    ],
}



In [44]:
def process_reaction_times(data):
    processed_data = {
        "scores": data["scores"],
        "reactionTimes": [rt["clickTimes"] for rt in data["reactionTimes"]],
        "reactionTimesGlow": [
            glow + [-1] * (4 - len(glow)) if len(glow) < 4 else glow[:4]
            for glow in data["reactionTimesGlow"]
        ],
    }
    df = pd.DataFrame(processed_data)

    # Flatten reactionTimes and reactionTimesGlow into separate columns
    reaction_df = pd.DataFrame(df["reactionTimes"].to_list(), columns=[
        f"reactionTime_{i+1}" for i in range(4)
    ])
    glow_df = pd.DataFrame(df["reactionTimesGlow"].to_list(), columns=[
        f"reactionTimesGlow_{i+1}" for i in range(4)
    ])
    return pd.concat([df.drop(columns=["reactionTimes", "reactionTimesGlow"]), reaction_df, glow_df], axis=1)

game_data = process_reaction_times(test_data)

In [45]:
game_data

,scores,reactionTime_1,reactionTime_2,reactionTime_3,reactionTime_4,reactionTimesGlow_1,reactionTimesGlow_2,reactionTimesGlow_3,reactionTimesGlow_4
0,2,1008,381,551,266,1348,1961,-1,-1
1,3,1007,999,499,245,1546,1460,-1,-1
2,0,897,787,300,456,1423,887,1186,-1
3,2,978,654,321,111,1423,111,-1,-1
4,2,700,800,600,200,700,-1,-1,-1
5,2,1500,500,700,100,3702,-1,-1,-1
6,1,1400,1300,800,900,1500,1200,1300,1400
7,2,1234,5678,4321,8765,900,-1,-1,-1


In [32]:
game_data.dtypes

scores                 int64
reactionTime_1         int64
reactionTime_2         int64
reactionTime_3         int64
reactionTime_4         int64
reactionTimesGlow_1    int64
reactionTimesGlow_2    int64
reactionTimesGlow_3    int64
reactionTimesGlow_4    int64
dtype: object